In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

2024-04-20 09:57:30.697187: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-20 09:57:30.972691: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 09:57:30.972717: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 09:57:31.021825: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-20 09:57:31.126012: I tensorflow/core/platform/cpu_feature_guar

In [2]:
train_set= ('./data_pre/spec_imgs/train')
test_set=('./data_pre/spec_imgs/test')
val_set= (r'./data_pre/spec_imgs/val')

In [3]:
train_datagen = image.ImageDataGenerator(rescale= 1./255)
val_datagen= image.ImageDataGenerator(rescale= 1./255)
test_datagen= image.ImageDataGenerator(rescale= 1./255)


In [4]:

train_generator = train_datagen.flow_from_directory(train_set,batch_size =512 ,class_mode = 'categorical')
test_generator = test_datagen.flow_from_directory(test_set,shuffle=True,batch_size =128 ,class_mode = 'categorical')
validation_generator = test_datagen.flow_from_directory(val_set,shuffle=True,batch_size =128 ,class_mode = 'categorical')

Found 699 images belonging to 10 classes.
Found 151 images belonging to 10 classes.
Found 149 images belonging to 10 classes.


In [5]:
x_train, y_train = next(train_generator)
x_val,y_val= next(validation_generator)
x_test, y_test = next(test_generator)

In [2]:
from tensorflow.keras.applications import InceptionV3

# Load the InceptionV3 model
model = InceptionV3(weights='imagenet', include_top=False)

# Print the summary of the model
model.summary()

87910968/87910968 [==============================] - 13s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv2d_94 (Conv2D)          (None, None, None, 32)       864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_94 (Ba  (None, None, None, 32)       96        ['conv2d_94[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_94 (Activ

In [8]:
import optuna
import pickle

from keras.applications import InceptionV3
from keras.models import Sequential
from keras.layers import Dense, Flatten
from tensorflow.keras.layers import Activation, BatchNormalization, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import swish, relu,selu


def objective(trial):
    # Define the search space for hyperparameters
    num_layers = trial.suggest_categorical('num_layers', [31, 62, 82])
    num_neurons = trial.suggest_int('num_neurons', 8, 10000)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)

    # Create the MLP model with hyperparameters
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the layers in the base model
    for layer in base_model.layers[:-num_layers]:
        layer.trainable = False

# Create a new model
    model = Sequential()

# Add the InceptionV3 base model to the new model
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(num_neurons, input_shape=(x_train.shape[1],)))
    model.add(Activation(relu))
    model.add(Dropout(0.2))
    
    model.add(Dense(10, activation='softmax'))

    # Compile the model with hyperparameters
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model with hyperparameters
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', min_delta=0.001, restore_best_weights=True)
    pruning_callback = optuna.integration.TFKerasPruningCallback(trial, 'val_loss')
    lr_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='min', min_lr=learning_rate/1000)
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=150, callbacks=[early_stopping, pruning_callback,lr_on_plateau], verbose=1)
    # Return the validation accuracy as the objective value
    return history.history['val_accuracy'][-1]

# Create an Optuna study and optimize the objective function
sampler = optuna.samplers.TPESampler(seed=42, n_startup_trials=100)
study = optuna.create_study(direction='maximize', sampler=sampler)

study.optimize(objective, n_trials=500)

save_path = './opt_study/optuna_incep_spec_r0_v2_study.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(study, f)

# Print the best hyperparameters and objective value
best_params = study.best_params
best_value = study.best_value
print('Best Hyperparameters:', best_params)
print('Best Objective Value:', best_value)


/home/alinnavas/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-04-20 09:57:51,056] A new study created in memory with name: no-name-5f25f442-a26c-4918-8590-dddac2802be2


Epoch 1/50


2024-04-20 09:57:52.428096: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-04-20 09:57:52.706383: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-20 09:57:55.320061: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-20 09:57:55.671028: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f5a5081d3d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-20 09:57:55.671052: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-04-20 09:57:55.683807: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1713603475.748409    3438 devic

4/4 [==============================] - 23s 493ms/step - loss: 5.7691 - accuracy: 0.0703 - val_loss: 3.6341 - val_accuracy: 0.1094
Epoch 2/50
4/4 [==============================] - 2s 422ms/step - loss: 3.1520 - accuracy: 0.1094 - val_loss: 2.3349 - val_accuracy: 0.0781
Epoch 3/50
4/4 [==============================] - 2s 416ms/step - loss: 2.2816 - accuracy: 0.1016 - val_loss: 2.3709 - val_accuracy: 0.1094
Epoch 4/50
4/4 [==============================] - 2s 422ms/step - loss: 2.2936 - accuracy: 0.1094 - val_loss: 2.3258 - val_accuracy: 0.1094
Epoch 5/50
4/4 [==============================] - 2s 421ms/step - loss: 2.2812 - accuracy: 0.1406 - val_loss: 2.3194 - val_accuracy: 0.0938
Epoch 6/50
4/4 [==============================] - 2s 419ms/step - loss: 2.2797 - accuracy: 0.0938 - val_loss: 2.3333 - val_accuracy: 0.0938
Epoch 7/50
4/4 [==============================] - 2s 422ms/step - loss: 2.2804 - accuracy: 0.1328 - val_loss: 2.3293 - val_accuracy: 0.1094
Epoch 8/50
4/4 [==============

[I 2024-04-20 09:58:36,224] Trial 0 finished with value: 0.078125 and parameters: {'num_layers': 0}. Best is trial 0 with value: 0.078125.


...

Epoch 1/50
4/4 [==============================] - ETA: 0s - loss: 5.5720 - accuracy: 0.1328

[I 2024-04-20 10:00:40,602] Trial 8 pruned. Trial was pruned at epoch 0.


Epoch 1/50


2024-04-20 10:00:41.737181: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at conv_ops_fused_impl.h:660 : UNKNOWN: CUDNN failed to allocate the scratch space for the runner or to find a working no-scratch runner.
[W 2024-04-20 10:00:41,752] Trial 9 failed with parameters: {'num_layers': 0} because of the following error: UnknownError().
Traceback (most recent call last):
  File "/home/alinnavas/.local/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_3108/616720336.py", line 39, in objective
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=50, callbacks=[early_stopping, pruning_callback], verbose=1)
  File "/home/alinnavas/.local/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/home/alinnavas/.local/lib/python3.10/site-packages/tensorflow/python/eag

UnknownError: Graph execution error:

Detected at node sequential_10/vgg16/block3_conv2/Relu defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/alinnavas/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/alinnavas/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/alinnavas/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 701, in start

  File "/home/alinnavas/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/alinnavas/.local/lib/python3.10/site-packages/ipykern